# Test Your Algorithm

## Instructions
1. From the **Pulse Rate Algorithm** Notebook you can do one of the following:
   - Copy over all the **Code** section to the following Code block.
   - Download as a Python (`.py`) and copy the code to the following Code block.
2. In the bottom right, click the <span style="color:blue">Test Run</span> button. 

### Didn't Pass
If your code didn't pass the test, go back to the previous Concept or to your local setup and continue iterating on your algorithm and try to bring your training error down before testing again.

### Pass
If your code passes the test, complete the following! You **must** include a screenshot of your code and the Test being **Passed**. Here is what the starter filler code looks like when the test is run and should be similar. A passed test will include in the notebook a green outline plus a box with **Test passed:** and in the Results bar at the bottom the progress bar will be at 100% plus a checkmark with **All cells passed**.
![Example](example.png)

1. Take a screenshot of your code passing the test, make sure it is in the format `.png`. If not a `.png` image, you will have to edit the Markdown render the image after Step 3. Here is an example of what the `passed.png` would look like 
2. Upload the screenshot to the same folder or directory as this jupyter notebook.
3. Rename the screenshot to `passed.png` and it should show up below.
![Passed](passed.png)
4. Download this jupyter notebook as a `.pdf` file. 
5. Continue to Part 2 of the Project. 

In [1]:
# replace the code below with your pulse rate algorithm.
import numpy as np
import scipy as sp
import scipy.io
import glob

import numpy as np
import scipy as sp
from scipy import signal
import scipy.io
from matplotlib import pyplot as plt
from matplotlib import mlab

fs=125 #sampling rate
minfreq=0.66 #40 bpm
maxfreq=4 #240

def LoadTroikaDataFile(data_fl):
    """
    Loads and extracts signals from a troika data file.

    Usage:
        data_fls, ref_fls = LoadTroikaDataset()
        ppg, accx, accy, accz = LoadTroikaDataFile(data_fls[0])

    Args:
        data_fl: (str) filepath to a troika .mat file.

    Returns:
        numpy arrays for ppg, accx, accy, accz signals.
    """
    data = sp.io.loadmat(data_fl)['sig']
    return data[2:]

def get_spectogram(sig):
    """
    Gets spectrogram and frequencies from given signal. 
    Calculate the magnitude of the acceleration.
    Arguments: 
        sig: required sensor signal (e.g. ppg or acc)
    Returns:
        spec: ndarray
        freqs: ndarray
    adapted from https://knowledge.udacity.com/questions/399609
    refered for understanding https://knowledge.udacity.com/questions/246546
    refered to https://vibrationresearch.com/blog/what-is-a-spectrogram/
    """
    spec, freqs, t=mlab.specgram(sig,NFFT=fs*8,Fs=fs,noverlap=6*fs, pad_to=10*fs)
    spec=spec[(freqs >= minfreq) & (freqs <= maxfreq)]
    freqs=freqs[(freqs >= minfreq) & (freqs <= maxfreq)]
    
    return spec, freqs

def BandpassFilter(sig): 
    """
    BandpassFilter function  allows certain frequency range only.
    Refer to ReadMe.md file for passband range.
    the Readme file assumes pulse rate will be restricted between 40BPM (beats per minute) and 240BPM
    Banpass Filter code has been adapted from Lowpassfilter in Lesson 3 
    Returns:
        Filtered signal
    """
    pass_band=(minfreq,maxfreq) 
    b, a = scipy.signal.butter(3, pass_band, btype='bandpass', fs=fs)
    return scipy.signal.filtfilt(b, a, sig)

def calcSNR(sig, est_freq):
    """
    Calculate Signal to noise ratio to determine how clean the signal is 
    Refered to Lesson 3 Exercise 3 Solution
    SNR was recommended to be used in https://knowledge.udacity.com/questions/399609
    """
    n = len(sig)*2
    #frequency of first harmonic
    harmonic_f = est_freq * 2
    
    # do Fourier Transform
    fft_mag = np.abs(np.fft.rfft(sig, n))
    freqs = np.fft.rfftfreq(n, 1/fs)
    
    #Find the frequencies close to heart rate and first harmonic of the heart rate
    window_f = 5/60
    fundamental_frequency_window = (freqs > est_freq - window_f) & (freqs < est_freq + window_f)
    harmonic_frequency_window = (freqs > harmonic_f - window_f) & (freqs < harmonic_f + window_f)
    
    #Calculate signal and noise power
    sig = np.sum(fft_mag[(fundamental_frequency_window) | (harmonic_frequency_window)])
    noise = np.sum(fft_mag[~ ((fundamental_frequency_window) | (harmonic_frequency_window))])
    snr = sig / noise
    
    return snr

def Featurize(ppg,accx, accy, accz,fs=fs):
    """Featurization of the accelerometer and ppg signal.
        Adapted from Lesson 4 .
      Args:
      accx: (np.array) x-channel of the accelerometer.
      accy: (np.array) y-channel of the accelerometer.
      accz: (np.array) z-channel of the accelerometer.
      fs: (number) the sampling rate of the accelerometer
      ppg:  (np.array) ppg signal

      Returns:
       n-tuple of accelerometer and ppg features
    """
    # Use Bandpass Filter to take out signals below 40 bpm and above 240 bpm
    ppg=BandpassFilter(ppg)
    accx=BandpassFilter(accx)
    accy=BandpassFilter(accy)
    accz=BandpassFilter(accz)
    
    #Get spectrogram frequencies
    spec_ppg, freqs_ppg = get_spectogram(ppg)
    spec_accx, freqs_accx = get_spectogram(accx)
    spec_accy, freqs_accy = get_spectogram(accy)
    spec_accz, freqs_accz = get_spectogram(accz)
      
    #Get frequency and time
    freqs = freqs_ppg.shape[0]
    t_steps = spec_ppg.shape[1]
    
    # Get indices for largest signal peaks
    ppg_ind = (-spec_ppg).argsort(axis=0)
    accx_ind = (-spec_accx).argsort(axis=0)
    accy_ind = (-spec_accy).argsort(axis=0)
    accz_ind = (-spec_accz).argsort(axis=0)
    
    return (freqs,t_steps,spec_ppg, freqs_ppg,
            ppg_ind,accx_ind,accy_ind,accz_ind)
           
def estimatePulseRate(ref_f1,ppg,accx, accy, accz,fs=fs): #refer to Lesson 4
    """Estimates the Pulse Rate of the accelerometer and ppg signal.
        Adapted from Lesson 4 .
        refered for understanding https://knowledge.udacity.com/questions/246546
      Args:
      accx: (np.array) x-channel of the accelerometer.
      accy: (np.array) y-channel of the accelerometer.
      accz: (np.array) z-channel of the accelerometer.
      fs: (number) the sampling rate of the accelerometer
      ppg:  (np.array) ppg signal

    Returns:
        errors, confidence numpy array
    """
    # get Ground Truth 
    # Load Reference
    groundtruth = scipy.io.loadmat(ref_f1)["BPM0"].reshape(-1)
    
    #Extract strongest frequencies in ppg and accelerometer signals 
    (freqs,t_steps,spec_ppg, freqs_ppg,
     ppg_ind,accx_ind,accy_ind,accz_ind) = Featurize(ppg,accx, accy, accz)
    
    # Estimate Pulse Rates adapted from https://knowledge.udacity.com/questions/399609
    #
    estimated_freq=[]
    for t in range(t_steps):
        for freq in range(freqs):
            i=0
            if freq == 2:# if just peaks at 2 frequencies just pick ppg
                estimated_freq.append(freqs_ppg[ppg_ind[freq][t]])
                break
            # try to remove accelerometer peaks to identify only ppg
            # we are just doing our best to remove peaks caused due to ARM movements
            elif np.all([(freqs_ppg[ppg_ind[freq][t]] != freqs_ppg[accx_ind[i][t]]), 
                      (freqs_ppg[ppg_ind[freq][t]] != freqs_ppg[accy_ind[i][t]]), 
                      (freqs_ppg[ppg_ind[freq][t]] != freqs_ppg[accz_ind[i][t]]),
                      (freqs_ppg[ppg_ind[freq][t]] != freqs_ppg[accx_ind[i+1][t]]),
                      (freqs_ppg[ppg_ind[freq][t]] != freqs_ppg[accy_ind[i+1][t]]),
                      (freqs_ppg[ppg_ind[freq][t]] != freqs_ppg[accz_ind[i+1][t]]),
                      (freqs_ppg[ppg_ind[freq][t]] != freqs_ppg[accx_ind[i+2][t]]),
                      (freqs_ppg[ppg_ind[freq][t]] != freqs_ppg[accy_ind[i+2][t]]),
                      (freqs_ppg[ppg_ind[freq][t]] != freqs_ppg[accz_ind[i+2][t]])]):
                estimated_freq.append(freqs_ppg[ppg_ind[freq][t]])
                break
    # use SNR from Lesson 3 to determine "purity" of the ppg signal this is what our confidence is
    confidence = []
    for i in range(len(estimated_freq)):
        snr = calcSNR(ppg, estimated_freq[i])
       # print("For {}, estimated_freq={} and snr={}".format(i, estimated_freq[i],snr))
        confidence.append(snr)
    
    predicted = np.array(estimated_freq) * 60
    
    #Get Error and Confidence in array
    error_arr = np.abs(groundtruth - predicted)
    conf_arr = np.array(confidence)
    #for i in range(len(conf_arr)):
    #    print("At index {}, confidence={} and error={}".format(i, estimated_freq[i],snr))
    return error_arr, conf_arr
    

def RunPulseRateAlgorithm(data_fl, ref_fl):
    # Load data using LoadTroikaDataFile
    ppg, accx, accy, accz = LoadTroikaDataFile(data_fl)
        
    # Compute pulse rate estimates and estimation confidence.
    errors, confidence = estimatePulseRate(ref_fl,ppg, accx, accy, accz)
    # Return per-estimate mean absolute error and confidence as a 2-tuple of numpy arrays.
    #errors, confidence = np.ones(100), np.ones(100)  # Dummy placeholders. Remove
    return errors, confidence